In [ ]:
import requests

# Overpass API endpoint to get boundary data (e.g., municipality boundaries)
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(
  relation["boundary"="administrative"]["name"="Gent"];
);
out body;
"""

# Send the query
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# Print the result (GeoJSON format)
print(data)


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import json

# Load district boundaries (GeoJSON)
districts = gpd.read_file("stadswijken-gent.geojson")

# Load and process population density (JSON)
with open("PopulationDensity.json", "r", encoding="utf-8") as file:
    population_data = json.load(file)

# Convert JSON to DataFrame
population_df = pd.DataFrame.from_dict(population_data["population_density_2024"],
                                       orient="index",
                                       columns=["population_density"])
population_df.index.name = "district_name"
population_df.reset_index(inplace=True)

# Merge population data with district geometry using "wijk" as the district identifier
districts = districts.merge(population_df, left_on="wijk", right_on="district_name", how="left")

# Load bus stop locations and routes (CSV)
bus_stops = pd.read_csv("bushaltes-gent.csv", delimiter=";")

# Convert bus stops to a GeoDataFrame using correct latitude & longitude
bus_stops_gdf = gpd.GeoDataFrame(bus_stops,
                                 geometry=gpd.points_from_xy(bus_stops["longitude"], bus_stops["latitude"]),
                                 crs="EPSG:4326")

# Plot the map
fig, ax = plt.subplots(figsize=(10, 8))
districts.plot(ax=ax, column="population_density", cmap="OrRd", legend=True, edgecolor="black", alpha=0.5)
bus_stops_gdf.plot(ax=ax, color="blue", markersize=10, label="Bus Stops")

# Add a basemap
ctx.add_basemap(ax, crs=districts.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

plt.legend()
plt.title("Ghent Bus Stops & Population Density")
plt.show()



In [ ]:
import geopandas as gpd

# Load the districts GeoJSON
districts = gpd.read_file("stadswijken-gent.geojson")

# Print the column names
print(districts.columns)

# Print the first few rows to inspect
print(districts.head())



In [ ]:
bus_stops = pd.read_csv("bushaltes-gent.csv", delimiter=";")
print(bus_stops.columns)  # Check the correct column names
print(bus_stops.head())    # Inspect the first few rows
 # Show first few rows to verify data
